# <font color="#332E3C"><b>Cost Prediction for Logistic Company</b></font>

>**Team MU**
>
>- Camilo Florez        (C0888810)
>- Syed Zumair          (C0885081)
>- Karman Singh Bains   (C0885084)
>- Mahaveersinh Chauhan (C0884854)
>- Joel Castro           (C0883863)
>- Manpreet Singh        (C0884727)


## Table of contents <a class="anchor" id="0.1"></a>

1. [Introduction](#1)
2. [Business Case Evaluation](#2)
3. [Data Identification](#3)<br>
4. [Prepare for Analysis](#4)<br>
    4.1 [Libraries](#4.1)<br>
    4.2 [General Functions](#4.2)
5. [Data Extraction](#5)
6. [Data Validation & Cleansing](#6)<br>
    6.1 [Duplicate Values](#6.1)<br>
    6.2 [Unique Values & Typographic Error](#6.2)<br>
    6.3 [Outliers](#6.3)<br>
    6.4 [Missing Values](#6.4)<br>
7. [Exploration Data Analysis](#7)
8. [Machine Learning Techniques](#8)<br>
    8.1[Split](#8.1)<br>
    8.2[Linear Regression](#8.2)<br>
    8.3[Random Forest Regressor](#8.3)<br>
    8.4[Gradient Boosting Regressor](#8.4)<br>
    8.5[XGB Booster](#8.5)
9. [Conclusion](#9)

[Back to Table of Contents](#0.1)
## Introduction <a class="anchor" id="1"></a>
As businesses increasingly rely on efficient and timely delivery of goods, logistics companies seek ways to optimize their delivery networks. They require accurate predictions of the costs associated with delivering packages across their network to achieve this. In this scenario, a logistics company has shared a dataset of network-wide deliveries and their corresponding trip costs. The task at hand is to build a model that can accurately predict the cost of deliveries, enabling the company to optimize its network and improve its overall efficiency.

[Back to Table of Contents](#0.1)
## Business Case Evaluation <a class="anchor" id="2"></a>

#### Problem definition
The prediction of the delivery cost is requested.

Evaluation Metric : RMSE Score

The file should contain a header and have the following format:
trip,cost
t333282728025, 23.3433

[Back to Table of Contents](#0.1)
## Data Identification <a class="anchor" id="3"></a>
The data contains information related to Logistic Company.<br><br>

|COLUMN| DESCRIPTION|
|:--|:--|
|Trip|Trip id|
|Date|Date, when trip was made|
|DayPart|Day or Night|
|ExWeatherTag|Heat/Snow|
|OriginLocation|Source location|
|DestinationLocation|Destination location|
|Distance|distance traveled|
|Type|Expedited or Not|
|Weight|weight carried by carriers|
|PackageType |Type of packages|
|carrier|Name of the carriers|


Datasets
Training data –Training dataset contains 38999 observations.
Actual responses (labels) are in the variable called ‘Cost’ Testing data – contains 802 observations with 11 variables and no labels

### Metadata 
Kaggel Competition<br>
**File descriptions**<br>
train.csv - the training Dataset<br>
test.csv - the test Dataset

[Back to Table of Contents](#0.1)
## Prepare for Data Analysis <a class="anchor" id="4"></a>

#### Import Libraries <a class="anchor" id="4.1"></a>

In [ ]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import plotly.express as px
import time
from tqdm import tqdm
from sklearn.ensemble import IsolationForest
from sklearn import metrics
from scipy import stats
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, SimpleImputer, IterativeImputer
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.cluster import DBSCAN
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import norm, skew
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

# Ignore warning messages
warnings.filterwarnings('ignore')

# More columns the line
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

# static variables
order_months = {"January":1, "February":2, "March":3, "April":4, "May":5, "June":6, "July":7, "August":8, "September":9, "October":10,
                "November":11, "December":12}


#### General Functions <a class="anchor" id="4.2"></a>

Class created for creating the graph and define the basic functionality of the graph.

In [ ]:
class Graphs:
    def __init__(self, title, x_rotation=None, x_fontsize=None, bar_rotation=None, bar_fontsize=None,
                 column_huge_name=None, legend_loc='upper left'):
        """Define Basic Functionality of the Graphs

        Args:
            title (String): Title of the Graph
            x_rotation (Int, optional): X Rotation of the Graph. Defaults to None.
            x_fontsize (Int, optional): Y Rotation of the Graph. Defaults to None.
            bar_rotation (Int, optional): Bar Rotation of the Graph. Defaults to None.
            bar_fontsize (Int, optional): Font size of the Graph. Defaults to None.
            column_huge_name (String, optional): Column Huge Name of the Graph. Defaults to None.
            legend_loc (String, optional): Legend Location of the Graph. Defaults to 'upper left'.
        """
        self.title = title
        self.x_rotation = x_rotation
        self.x_fontsize = x_fontsize
        self.bar_rotation = bar_rotation
        self.bar_fontsize = bar_fontsize
        self.column_huge_name = column_huge_name
        self.legend_loc = legend_loc
        
    def bar_plot(self, column_x, column_y, column_x_name, column_y_name, column_hue=None, order_x=None):
        """Plot Bar Graph
        Args:
            column_x (DataFrame): X-Bar of the Graph
            column_y (DataFrame): Y-Bar of the Graph
            column_x_name (String): Name of the X-Bar
            column_y_name (String): Name of the Y-Bar
            column_hue (String, optional): Column Hue of the Graph. Defaults to None.
            order_x (List of the String, optional): Order to plot the categorical levels in the Graph. Defaults to None.
        """
        fig, ax = plt.subplots(figsize=(15, 7))
        sns.barplot(x=column_x, y=column_y, hue=column_hue, order=order_x, ax=ax, errorbar=None)
        [ax.bar_label(cont, padding=3, fmt='%d', fontsize=self.bar_fontsize, rotation=self.bar_rotation) for cont in
         ax.containers]
        plt.xlabel(column_x_name)
        plt.ylabel(column_y_name)
        if column_hue is not None and self.column_huge_name is not None:
            plt.legend(title=self.column_huge_name, loc=self.legend_loc)
        plt.title(self.title, fontsize=12).set_position([0.5, 1.05])
        plt.xticks(rotation=self.x_rotation, fontsize=12)
        plt.show()

    @staticmethod
    def distribution_plot(data_filter, cols_num):
        """Plot the Distribution Graph

        Args:
            data_filter (DataFrame): Data for the Graph
            cols_num (List of String): Columns list for the Graph
        """
        fig, axes = plt.subplots(ncols=3, figsize=(16, 4))
        fig.subplots_adjust(hspace=1)
        for i, col in enumerate(cols_num):
            sns.kdeplot(data=data_filter,x=col, shade=True, kernel='gau',color="Red", ax=axes[i], legend=True)
            axes[i].set_title(col)
            fig.tight_layout()

    def heat_map_corr(self, data):
        """Correlation heat map of the graph

        Args:
            data (DataFrame): Data for the Graph
        """
        plt.figure(figsize=(12, 8))
        plt.title(self.title, y=1.05,size=12)
        sns.heatmap(data, square=True, cmap='RdYlBu', linecolor="white", annot=True,annot_kws={"size": 7})
        plt.tight_layout()
        plt.show()

    def heatmap_nulls(self, data):
        """Graph of the Null value

        Args:
            data (DataFrame): Data for the Graph
        """
        ax = plt.axes()
        ax.xaxis.tick_top()
        sns.heatmap(data.isnull(), cbar=False, ax=ax)
        plt.title(self.title)
        plt.xlabel("Column")
        plt.ylabel("Quantity")
        plt.xticks(fontsize=9)
        plt.yticks(fontsize=6)
        plt.show() 
    

Function created for making the dummy variable of the categorical data.

In [ ]:
def merge_dummy_data(data, column):
    """Create Dummy variable of the data and merge to original dataframe

    Args:
        data (DataFrame): Original Data
        column (String): Categorical Column of the Data

    Returns:
        DataFrame: Return Merged DataFrame
    """
    dummies = pd.get_dummies(data[column], prefix=column, dummy_na=False)
    merged_imputed = pd.concat([data, dummies], axis=1)
    merged_imputed.drop([column], axis=1, inplace=True)
    return merged_imputed

Function created to get the missing values

In [ ]:
def missing_value(train):
   """Check Missing Value
   Args:
       train (DataFrame): Data to check missing value

   Returns:
       DataFrame: Return DataFrame which has summery of the missing value
   """
   obs = train.isnull().sum().sort_values(ascending = False)
   percent = round(train.isnull().sum().sort_values(ascending = False)/len(train)*100, 2)
   missing = pd.concat([obs, percent], axis = 1,keys= ['Number of Observations', 'Percent'])
   return missing

Function created to identify the outliers

In [ ]:
def iden_outliers(df,cols_num):
    """Identify The Outlier of the Data

    Args:
        df (DataFrame): Data to find the outlier
        cols_num (String): Find the outlier for that particular column
    """
    fig, ax = plt.subplots(nrows=3,ncols=1,figsize=(12,6))
    fig.subplots_adjust(hspace=1)
    for i, col in enumerate(cols_num):
        sns.boxplot(x=col,data=df,ax=ax[i])
        #ax[i].set_title(col)

Function created to analysis outliers

In [ ]:
def outliers_analysis(df):
    """Get the Mean, Mode, Median and Standard Deviation of the data

    Args:
        df (DataFrame): Data that need to analysis the outlier
    """
    mean = np.mean(df_train_date[df])
    median = np.median(df_train_date[df])
    mode = stats.mode(df_train_date[df])
    std_dev = np.std(df_train_date[df])
    print("mean:",mean)
    print("median:",median)
    print("mode:",mode)
    print("standar desviation:",std_dev)

Function created for removing outlier using IQR algorithm

In [ ]:
def iqr_outliers(df):
    """Apply the IQR outlier algorithm to remove the outlier

    Args:
        df (DataFrame): Data that need to remove outlier

    Returns:
        DataFramr: Filterd DataFrame
    """
    out = []
    if not any(df):
        return out
    q1 = df.quantile(0.25)
    q3 = df.quantile(0.75)
    iqr = q3 - q1
    lower_tail = q1 - 1.5 * iqr
    upper_tail = q3 + 1.5 * iqr
    for i in df:
        if i > upper_tail or i < lower_tail:
            out.append(i)
    return out

Function created to transfor the categorical columns to numerical columns

In [ ]:
def format_data_etiqueta(df,c):
    """Apply Label Encoder

    Args:
        df (DataFrame): Dataset
        c (Column): Column that needs to apply label encoder
    """
    le = LabelEncoder()
    df[c]= le.fit_transform(df[c])
    

Linear Regression Fuction

In [ ]:
def linearregresion (X_train,y_train):
    """Apply Linear Regression Alogrithm

    Args:
        X_train (DataFrame): Train Dataset
        y_train (DataFrame): Target Dataset of the Train Dataset

    Returns:
        Object: Object of the Linear Regression Algorithm
    """
    reg = LinearRegression().fit(X_train, y_train)
    return reg

Random forest regressor function

In [ ]:
def random_forest_regressor(x_train,y_train):
    """Apply Random Forest Regression Algorithm

    Args:
        X_train (DataFrame): Train Dataset
        y_train (DataFrame): Target Dataset of the Train Dataset

    Returns:
        Object: Object of the Linear Regression Algorithm
    """
    try:
        rfr = RandomForestRegressor(random_state=0)
        rfr.fit(x_train, y_train)
        return rfr
    except Exception as ex:
        return None

Function created for get the data metrics 

In [ ]:
def model_error(y_test,y_pred):
    """Check the model performance

    Args:
        y_test (DataFrame): Test Dataset
        y_pred (DataFrame): Predicted Dataset
    """
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    performance = r2 - mse
    print("performance: ", performance)
    print("MAE: ", mae)
    print("R2: ", r2)
    print("MSE: ", mse)
    print("RMSE: ", np.sqrt(mse))

Function created to find the best hyperparameters using GridSearchCV

In [ ]:
def grid_search_cv(model: object, param_grid: dict, X_train: object, y_train: object) -> object:
    """Get the best hyperparameter to the model

    Args:
        model (object): initial model
        param_grid (dict): hyperparameters dict
        X_train (object): Train Dataset
        y_train (object): Target Dataset

    Returns:
        object: Model with Best estimator
    """
    gs = GridSearchCV(model, param_grid, cv=3, scoring="neg_mean_squared_error", refit=True, n_jobs= -1) 
    gs.fit(X_train, y_train)
    best_params = gs.best_params_
    best_estimators = gs.best_estimator_

    # Print the best params
    print('Best Params: ', best_params)
    # Print the best estimators
    print('Best Estimators', best_estimators)
    # Print the accuracy score
    print('Accuracy Score:', best_estimators.score(X_train, y_train))
    
    return best_estimators

Function created to find best hyperparameter using RandomizedSearchCV

In [ ]:
def randomized_search_cv(model: object, param_grid: dict,  X_train: object, y_train: object, n_iter: int = 20, verbose: int = 2) -> object:
    """Get the best hyperparameter to the model

    Args:
        model (object): initial model
        param_grid (dict): hyperparameters dict
        X_train (object): Train Dataset
        y_train (object): Target Dataset
        n_iter (int, optional): number of iterations. Defaults to 25.
        verbose (int, optional): verbose. Defaults to 2.

    Returns:
        object: model with best estimators
    """
    rnd_search_cv = RandomizedSearchCV(model, param_grid, n_iter=n_iter,scoring='neg_mean_absolute_error',verbose=verbose,cv = 3, n_jobs=-1)
    rnd_search_cv.fit(X_train, y_train)
    # Get the best estimator on Randomized SearchCV
    best_estimators = rnd_search_cv.best_estimator_

    # Print the best estimators
    print('Best Estimators', best_estimators)    
    # Print the accuracy score
    print('Accuracy Score:', rnd_search_cv.score(X_train, y_train))
    
    return best_estimators

Function created to identify the importance columns (Model with coef)

In [ ]:
def feature_importance_lineal(model: object, X_train: object) -> None:
    """Get the list of feature importance

    Args:
        model (object): initial model
        X_train (object): Train Dataset
    """
    for name, score in sorted(zip(X_train.columns, model.coef_), key=lambda x: x[1], reverse=True):
        if score == 0:
            break
        print("Column:", name, "\t Score:", score)

Function created to identify the importance columns (Model with feature importances)

In [ ]:
def feature_importance(model: object, X_train: object) -> None:
    """Get the list of feature importance

    Args:
        model (object): initial model
        X_train (object): Train Dataset
    """
    for name, score in sorted(
            zip(X_train.columns, model.feature_importances_), key=lambda x: x[1], reverse=True): 
        if score == 0:
            break
        print("Column:", name, "\t Score:", score)

Function to export the results

In [ ]:
def export_result (y,df,name):
    """Save CSV File of the predicted result

    Args:
        y (DataFrame): Predicted Value of the Data
        df (DataFrame): Original Test DataFrame
        name (String): Name of the CSV File
    """
    result = pd.DataFrame(y, columns=["cost"])
    df_test_target = pd.concat([result, df["trip"]], axis=1)
    test = df_test_target[["trip", "cost"]]
    test.to_csv(name + ".csv", index=0)

Function to get the Epoch time

In [ ]:
def date_to_epoch(date):
    """Epoch Time

    Args:
        date (object): Date parameter
   """
    epoch = datetime.utcfromtimestamp(0)
    return int((date - epoch).total_seconds())

##### [Back to Table of Contents](#0.1)
## Data Extraction <a class="anchor" id="5"></a>
Read the train and test Datasets.

In [ ]:
# Reading the training and test dataset
df_train = pd.read_csv('/kaggle/input/cost-prediction-for-logistic-company-2023w-aml1413/train.csv')
df_test = pd.read_csv('/kaggle/input/cost-prediction-for-logistic-company-2023w-aml1413/test.csv')

Get the first 5 and the last 5 rows of the train dataset to have an idea of how the data is presented.

In [ ]:
df_train.head()

In [ ]:
df_train.tail()

According to the first and last 5 columns, there are some NaN data to clean.

Get the shape of train and test dataset

In [ ]:
print('The shape of the train dataset and test dataset is:', df_train.shape, df_test.shape)

Get the basic datasets information to check the data types.

In [ ]:
print("Train Info")
df_train.info()

In [ ]:
print("Test Info")
df_test.info()

#### Data Types
From the above:

There are 8 Categorical features:

trip:                Ordinal category<br>
dayPart:             Binary category<br>
exWeatherTag:        Binary category<br>
originLocation:      Ordinal category<br>
type:                Binary category<br>
packageType:         Ordinal category<br>
carrier:             Nominal category<br>
destinationLocation: Ordinal category

There is 1 Date type

There are 3 numeric features:
weight:   Numeric Discret<br>
cost:     Numeric Continuous<br>
distance: Numeric Continuous<br>

[Back to Table of Contents](#0.1)
## Data Validation & Cleansing <a class="anchor" id="6"></a>

#### Describe

Initial analysis to identify minimum, maximum, standard, and quartile values for each column. There are important data to take into account, such as:
- There are two columns with unique value.
- The mean price is 80.97 for every delivery. However, the min and max prices show possible outliers affecting the mean.
- Possibly there are misleading values according to the number of records in the columns, type, packagetype and exweathertag.

In [ ]:
df_train.describe(include="all").T

In [ ]:
df_test.describe(include="all").T

#### Date attribute is transformed

The date column is changed to date format, the Day,Month and epoch time columns are also created. 

In [ ]:
# copy of the data train and data test are created
df_train_date=  df_train.copy()
df_test_date=  df_test.copy()

# Set Date column as a datetime data type
# Train Dataset
df_train_date["date"] = df_train["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df_train_date["date"] = df_train["date"].astype("datetime64[ns]")
df_train_dat = np.vstack(df_train['date'].astype(str).apply(lambda x:list(map(int,x.split('-')))).values)

df_train_date['Year'] = df_train_dat[:,0]
df_train_date['Monthn']= df_train_dat[:,1]
df_train_date['Dayn'] = df_train_dat[:,2]
df_train_date['Day'] = df_train_date["date"].dt.day_name()
df_train_date['Month'] = df_train_date["date"].dt.month_name()
df_train_date['epoch_times'] = df_train_date['date'].apply(lambda x: date_to_epoch(x))

# Test Dataset
df_test_date["date"] = df_test["date"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df_test_date["date"] = df_test["date"].astype("datetime64[ns]")
df_test_dat = np.vstack(df_test['date'].astype(str).apply(lambda x:list(map(int,x.split('-')))).values)

df_test_date['Year'] = df_test_dat[:,0]
df_test_date['Monthn']= df_test_dat[:,1]
df_test_date['Dayn'] = df_test_dat[:,2]
df_test_date['Day'] = df_test_date["date"].dt.day_name()
df_test_date['Month'] = df_test_date["date"].dt.month_name()
df_test_date['epoch_times'] = df_test_date['date'].apply(lambda x: date_to_epoch(x))


The change made is verified

In [ ]:
df_train_date

#### Duplicate Values <a class="anchor" id="6.1"></a>
No duplicate values


In [ ]:
df_train_date[df_train_date["trip"].isin(df_train_date[df_train_date.duplicated()]["trip"].values)]

#### Unique Values and typographic error <a class="anchor" id="6.2"></a>

There are no typographical errors or redundancy in the name of the variables. It is observed the column type and packagetype have only 1 level or value, so they must be taken into consideration and evaluated.

In [ ]:
cols_cat = ['dayPart', 'exWeatherTag', 'originLocation','destinationLocation', 'carrier', 'type', 'packageType']
for col in cols_cat:
    print(f'Column {col}:{df_train_date[col].nunique()} categories')
    print(df_train_date[col].value_counts().index.tolist())


#### Identify Outliers <a class="anchor" id="6.3"></a>

The identification of outliers is carried out by means of box plots and the histogram for the numerical variables.
Boxplots are a useful graphical tool for identifying outliers,show the distribution of data in terms of the median, quartiles, and extreme values.

In [ ]:
cols_num = ['cost','distance','weight']
iden_outliers(df_train_date,cols_num)

Distribution plots serve to complement the information provided by a boxplot. A distribution plot shows the shape and symmetry of the data distribution, which can be helpful in identifying patterns that are not apparent on a boxplot.

In [ ]:
Graphs.distribution_plot(df_train_date,cols_num)

Within the analysis developed at this point of the numerical variables Cost, distance and weight, the following observations can be made:

Cost

###### - The mean (80.97) is greater than the median (46.47), which suggests that the distribution be skewed to the right.
###### - The mode of the distribution is 63, which means that this value is the most common in the sample. This value is not equal to the mean or median, which also suggests an asymmetry in the distribution.
###### - The standard deviation (180.71) is relatively high compared to the mean. It may have outliers.

In [ ]:
print("Analysis cost")
outliers_analysis("cost")

Weight

###### - The mean (42.29) is greater than the median (25), which suggests that the distribution be skewed to the right.
###### - The mode (mode) is much smaller than the mean and median. which also suggests an asymmetry in the distribution. 
###### - The standard deviation (180.71) is relatively high compared to the mean.It may have outliers.

In [ ]:
print("Analysis weight")
outliers_analysis("weight")

Distance

###### - For this case, the data suggest that most of the samples are within the range of 1,000 to 3,000 kilometers, with a mean and median that are close to that range. The fact that the mode is also in this range indicates that there is a high concentration of values around this distance.

In [ ]:
print("Analysis distance")
outliers_analysis("distance")

##### As we see above, Cost, Distance and Weight column has outlier. But these all columns are related to target variable so we can not remove it.

#### Ouliers imputation

This part is for experimentation.

Based on general observations, we might consider using outlier detection techniques that focus on identifying outliers in the right tail of the distribution for the cost, and weight columns, since the mean is greater than the median and the distribution is skewed to the right. Therefore, the technique "interquartile range rule (IQR)" could be used. This technique is useful when the distribution is not symmetric and has long tails in one direction.


In [ ]:
df_train_without_Outliers =  df_train_date.copy()

We called the function to find the outliers with the IQR technique

In [ ]:
outliers_cost = iqr_outliers(df_train_without_Outliers["cost"])
outliers_weight = iqr_outliers(df_train_without_Outliers["weight"])

The imputation of the outliers was made

In [ ]:
#calculate the median
median_cost = np.median(df_train_without_Outliers['cost'])
median_cost = np.median(df_train_without_Outliers['weight'])
# Impute outliers with the median
df_train_without_Outliers['cost'].replace(outliers_cost, median_cost, inplace=True)
df_train_without_Outliers['weight'].replace(outliers_weight, median_cost,inplace=True)



A distribution graph was made to see the behavior after imputing the outliers.

In [ ]:
Graphs.distribution_plot(df_train_without_Outliers,cols_num)

#### Missing Values <a class="anchor" id="6.4"></a>

This step identifies the percentage and quantity of missing values for each column. For this case, the columns type, packagetype and exWeathertag have 93.52%, 90.90% and 87.41% of missing values.

In [ ]:
# Missing Values train
missing_value(df_test_date)

In [ ]:
# Missing Values train
missing_value(df_train_date)

Function heatmap_nulls is created to show a heat map about nulls in all data frames. With this process, the goal is to identify the columns that are not relevant to the business problem.

In [ ]:
Graphs("Quantity Nulls per Column").heatmap_nulls(df_train_date)

A copy of the dataset was created to analyze the data

In [ ]:
df_EDA = df_train_date.copy()

##### 'Type' Column - We fill 'Not' in the missing value. Because according to the dataset discription we have only two category for the Type column.

In [ ]:
# type variable
df_train_without_Outliers['type'] = df_train_without_Outliers['type'].fillna('Not')
df_train_date['type'] = df_train_date['type'].fillna('Not')
df_EDA['type'] = df_train_date['type'].fillna('Not')
df_test_date['type'] = df_test_date['type'].fillna('Not')

##### 'packageType' Column - We fill 'standard' in the missing value.


In [ ]:
# packagetype variable
df_train_without_Outliers['packageType'] = df_train_without_Outliers['packageType'].fillna('standar')
df_train_date['packageType'] = df_train_date['packageType'].fillna('standar')
df_EDA['packageType'] = df_train_date['packageType'].fillna('standar')
df_test_date['packageType'] = df_test_date['packageType'].fillna('standar')

##### 'exWeatherTag' Column - According the dataset, only 6(June) month have 'heat' value in the dataset. And 12(december) month have 'snow' value in the dataset. So we assume that from the January to June we put 'heat' and from the July to December we put 'snow'.


In [ ]:
# exweather variable
df_train_without_Outliers['exWeatherTag'] = df_train_date['Monthn'].apply(lambda x: 'heat' if x<= 6 else 'snow') 
df_train_date['exWeatherTag'] = df_train_date['Monthn'].apply(lambda x: 'heat' if x<= 6 else 'snow') 
df_EDA['exWeatherTag'] = df_train_date['Monthn'].apply(lambda x: 'heat' if x<= 6 else 'snow') 
df_test_date['exWeatherTag'] = df_train_date['Monthn'].apply(lambda x: 'heat' if x<= 6 else 'snow')

In [ ]:
df_EDA

[Back to Table of Contents](#0.1)
###  Exploration Data Analysis <a class="anchor" id="7"></a>

Convert categorical variables to numeric

In [ ]:
cat_format = ['dayPart','exWeatherTag','originLocation','destinationLocation','type','packageType','carrier']
for var in cat_format:
    format_data_etiqueta(df_EDA,var)
    format_data_etiqueta(df_EDA,var)


#### Correlation of Features
According to the correlation matrix of the dataset, different variables with high correlations can be evidenced, such as: the distance variable with originLocation and destinationLocation. With respect to our target, this has a correlation of 0.86 with the weight and -0.66 with packageType, also a minor correction of 0.23 with the carrier variable.

In [ ]:
Graphs("Correlation of Features").heat_map_corr(df_EDA.corr())

We plot a list of the variables from highest to lowest correlation with the target

In [ ]:
corr = df_EDA.corr()
corr[['cost']].sort_values(by = 'cost',ascending = False)\
.style.background_gradient()

##### Graph: Distance and (Mean cost)
The graph is made in order to know the relationship between distance and cost. As can be seen, the variation of the distance does not have a great significance within the cost.

In [ ]:
df_graph = df_train_date[["cost", "distance"]]
df_graph = df_graph.groupby([ "distance"], as_index=False)["cost"].mean()
Graphs("Distance (Mean cost) by year", bar_fontsize=7, x_rotation=0, bar_rotation=0,
              column_huge_name="Year", legend_loc="best").bar_plot(df_graph['distance'],
                                                                           df_graph['cost'],
                                                                           "distance", "cost")

##### Graph: Media Cost by Month and year
In order to obtain data about the variation of the cost in the course of the months and years, the graph is made. The graph shows a variation in the price in the different months, being January the month with the highest cost, in turn it can be seen that as the years go by, the cost increases between 2 and 3%

In [ ]:
df_graph = df_train_date[["cost", "Month",'Year']]
df_graph = df_graph.groupby(["Month",'Year'], as_index=False)["cost"].mean()
df_graph.sort_values(by="Month", inplace=True, key=lambda x: x.map(order_months))
fig = px.line(df_graph, x="Month", y="cost", color = 'Year')
fig.update_layout(title=dict(text="Cost (mean) per year", font=dict(size=20), yref='paper'))
fig.show()

#### Cost (mean) vs dayPart per Year

This graph is highlighted in order to observe the relationship over the years between the cost and daypart variables. There is no evidence of a significant relationship or a variation in the cost more than the one that normally increases per year.


In [ ]:
df_graph = df_train_date[["cost", "dayPart",'Year']]
df_graph = df_graph.groupby(["dayPart",'Year'], as_index=False)["cost"].mean()
df_graph.sort_values(by="dayPart", inplace=True, key=lambda x: x.map(order_months))
fig = px.bar(df_graph, x="dayPart", y="cost", color = 'Year')
fig.update_layout(title=dict(text="Cost (mean) vs dayPart per Year", font=dict(size=20), yref='paper'))
fig.show()

##### Carrier (Mean cost) by Month 
Cost of the Carrier Type 'C' is very high as compared to the other Types.
The behavior of the variables in the different months is similar with the exception of variable C, which stands out for having the highest cost mean.

In [ ]:
df_graph = df_EDA[["cost", "carrier", "Month"]]
df_graph = df_graph.groupby(["carrier", "Month"], as_index=False)["cost"].mean()
Graphs("Carrier (Mean cost) by Month", bar_fontsize=7, x_rotation=0, bar_rotation=0,
              column_huge_name="Year", legend_loc="best").bar_plot(df_graph['Month'],
                                                                           df_graph['cost'],
                                                                           "Month", "cost", df_graph["carrier"],order_months)

##### Weight(mean) vs packageType
According to the graph, there is a relationship between the packageType and weight column where the highest concentration of data is found in the TT variable with an average weight of 302.

In [ ]:
df_graph = df_train_date[["weight", "packageType"]]
df_graph = df_graph.groupby(["packageType"], as_index=False)["weight"].mean()
Graphs("Weight vs packageType").bar_plot(df_graph['packageType'], df_graph['weight'], "packageType", "weight")

##### Distribution Between Cost and Weight by type
This graph is made in order to see the relationship between the variables weight cost and type. In this graph, two different behaviors are observed, one linear and the other exponential, the latter starting from the weight of 200. In turn, it is observed that the type variable has a fairly close relationship between the exponential graph. From the above, it can be deduced that there is a close relationship between the weight and cost variable without mentioning that the type variable has a particular behavior in the price increase.

In [ ]:
fig_px = px.scatter(df_train_date, "weight", "cost",color = "type",title="Distribution Between Cost and Weight by type ")
fig_px.update_coloraxes(showscale=False)
fig_px.show()

##### Carrier (Mean cost) by exWeatherTag.
Cost of the Carrier Type ‘C’ is very high as compared to the other Types. Also the relation of weathertag is not changing with the carrier column 

In [ ]:
df_graph = df_train_date[["cost", "exWeatherTag", "carrier"]]
df_graph = df_graph.groupby(["exWeatherTag", "carrier"], as_index=False)["cost"].mean()
Graphs("carrier (Mean cost) by exWeatherTag", bar_fontsize=9, x_rotation=0, bar_rotation=0,
              column_huge_name="Weather", legend_loc="best").bar_plot(df_graph['carrier'],
                                                                           df_graph['cost'],
                                                                           "carrier", "cost", df_graph["exWeatherTag"])

In [ ]:
del df_EDA

The initial correlation is created to understand which fields are valuable for solving the problem definition (cost). Initially, it is notable that year, parking assist, alloy wheels, and transmission have a bit of relevance. Nevertheless, the other columns are categorical. Those could be more accurate once they become to numerical. As a result, it is necessary cleaning the data.

[Back to Table of Contents](#0.1)
### Machine Learning Techniques <a class="anchor" id="8"></a>

There is a defined linear regression,Random Fores Regressor,Gradient Boosting Regressor and XGB Regressor to check the accuracy.

##### We have dropped the unnecessary column for the training the model

In [ ]:
X1 = df_train_date.drop(columns=['cost','date','trip','Day','Month','Dayn','Monthn','Year','dayPart',
                                 'Dayn','Monthn','type','originLocation','destinationLocation'])
y1 = df_train_date['cost']
t1 = df_test_date.drop(columns=['date','trip','Day','Month','Year','dayPart',
                                'Dayn','Monthn','Dayn','Monthn','type','originLocation','destinationLocation'])

Check the dataframe 

In [ ]:
X1

##### Create the dummy variable of the categorical data.

In [ ]:
#Format Data

#Train
cat_format_onehot = ['carrier','exWeatherTag','packageType']
#cat_format_onehot = ['originLocation','destinationLocation','type','packageType','carrier','exWeatherTag']
for var in cat_format_onehot:
    X1 = merge_dummy_data(X1,var)

#Test
for var in cat_format_onehot:
    t1 = merge_dummy_data(t1,var)

In [ ]:
X1

##### Train and Test Split <a class="anchor" id="8.2"></a>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.25, random_state = 10)

Check the number of columns

In [ ]:
print('The shape of the train dataset and test dataset is:', X1.shape, t1.shape)

### Lineal Regression <a class="anchor" id="8.2"></a>

Linear regression is a statistical technique used to measure the relationship between a dependent variable and one or more independent variables. It is used to create a model that can be used to make predictions, such as predicting a company’s stock price based on various factors. Linear regression is useful for understanding the strength and direction of the relationship between variables and can help identify which independent variables are most strongly associated with the dependent variable.

##### Initial Parameters

Firstly, it is necessary to identify what are the best hyperparameter to improve our model.

In [ ]:
param_grid_LN = {'fit_intercept': [True, False],
              'copy_X': [True, False],
              'n_jobs': [1, 2, 4, 8, 16,32,48,200],
              'positive': [True, False]}

LN = LinearRegression()


#### Grid Search
Grid search involves specifying a grid of potential values for each hyperparameter, and then exhaustively searching through the grid to determine the combination of hyperparameters that yields the best performance of the model on a validation set.

In [ ]:
best_estimators_LN = grid_search_cv(LN, param_grid_LN, X_train, y_train)
LNG = best_estimators_LN
LNG.fit(X_train, y_train)
LNG_y_pred = LNG.predict(X_test)


In [ ]:
print('Grid Search - Linear Regression')
model_error(y_test,LNG_y_pred) 


#### Normal Execution

In [ ]:
reg = linearregresion(X_train, y_train )
LN_y_pred = reg.predict(X_test)
print('Normal - Linear Regressor')
model_error(y_test,LN_y_pred)

#### Feature Importance
In this section, Transaction4, and SumTransactions have great relevance in the prediction. With this in mind, creating other groups to expand the combinations to improve the model is possible.

In [ ]:
feature_importance_lineal(LNG, X_train)

In [ ]:
feature_importance_lineal(reg, X_train)

#### Export Results

In [ ]:
LNG_y_pred_f = LNG.predict(t1)
LN_y_pred_f = reg.predict(t1)
export_result(LNG_y_pred_f,df_test,"Test15 LN - Gridsearch")
export_result(LN_y_pred_f,df_test,"Test15 LN - Normal")

##### Graph Error
The scatter graph of the error is made to know its behavior.

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(12, 4))
axes[0].scatter(y_test, LNG_y_pred)
axes[1].scatter(y_test, LN_y_pred)
axes[0].set_title('Linear Regression - Grid search', fontsize=12)
axes[1].set_title('Linear Regression - Normal', fontsize=12)
axes[0].set_xlabel('y_test', fontsize=12)
axes[0].set_ylabel('y_pred', fontsize=12)
axes[1].set_xlabel('y_test', fontsize=12)
axes[1].set_ylabel('y_pred', fontsize=12)
fig.tight_layout()

### Export Results
In this section, the results is exported to compare to kaggle competition

Kaggle Score: 0.59628
<br/>
Local Score: 0.618680

### Random Fores Regressor <a class="anchor" id="8.3"></a>
Random Forest Regressor is a machine learning algorithm used to predict continuous values. It is commonly used to solve business problems such as predicting future prices or costs. Random Forest Regressor is a powerful algorithm that combines multiple decision trees to create an ensemble model. It can handle missing values, outliers, and high-dimensional data, making it suitable for a wide range of problems.

##### Initial Parameters

Firstly, it is necessary to identify what are the best hyperparameter to improve our model.

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2010, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# First create the base model to tune
RFR = RandomForestRegressor()

#### Randomized Search CV

In [ ]:
best_estimators = randomized_search_cv(RFR, random_grid, X_train, y_train)
RFR_R = best_estimators
RFR_R.fit(X_train, y_train)
RFR_R_y_pred = RFR_R.predict(X_test)


In [ ]:
print('Randomized Search - Random Fores Regressor')
model_error(y_test,RFR_R_y_pred) 

#### Normal Execution

In [ ]:
RFR_N = random_forest_regressor(X_train, y_train)
RFR_N_y_pred = RFR_N.predict(X_test)
print('Normal - Random Fores Regressor')
model_error(y_test,RFR_N_y_pred)

#### Feature Importance

In [ ]:
feature_importance(RFR_R, X_train)

In [ ]:
feature_importance(RFR_N, X_train)

#### Export Results

In [ ]:
RFR_R_y_pred_f = RFR_R.predict(t1)
RFR_N_y_pred_f = RFR_N.predict(t1)
export_result(RFR_R_y_pred_f,df_test,"Test15 RFR - Randomized")
export_result(RFR_N_y_pred_f,df_test,"Test15 RFR - Normal")

#### Graph Error

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(16, 4))
axes[0].scatter(y_test,RFR_R_y_pred)
axes[1].scatter(y_test,RFR_N_y_pred)
axes[0].set_title('Random Fores Regresso - Randomized Search CV', fontsize=12)
axes[1].set_title('Random Fores Regresso - Normal', fontsize=12)
axes[0].set_xlabel('y_test', fontsize=12)
axes[0].set_ylabel('y_pred', fontsize=12)
axes[1].set_xlabel('y_test', fontsize=12)
axes[1].set_ylabel('y_pred', fontsize=12)
fig.tight_layout()

### Gradient Boosting Regressor <a class="anchor" id="8.4"></a>

Gradient Boosting Regressor is a machine learning algorithm used to build regression models that are capable of capturing nonlinear relationships between the target variable and features. It can handle missing values, outliers, and high cardinality categorical features without any special preprocessing. Gradient Boosting Regressor is a powerful algorithm that can achieve high accuracy and has good usability for a wide range of problems.

##### Initial Parameters

Firstly, it is necessary to identify what are the best hyperparameter to improve our model.

In [ ]:
param_grid_GBR = {
    'n_estimators': [50, 100,500],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [2, 3, 4],
    'min_samples_split': [2, 5, 10],
}

# First create the base model to tune
GBR = GradientBoostingRegressor()

#### Grid search

In [ ]:
bestest_GBR = grid_search_cv(GBR, param_grid_GBR, X_train, y_train)
GBR_G = bestest_GBR
GBR_G.fit(X_train, y_train)
GBR_G_y_pred = GBR_G.predict(X_test)

In [ ]:
print('Grid Search - GBR Regression')
model_error(y_test,GBR_G_y_pred) 

#### Normal Execution

In [ ]:
GBR_N = GBR.fit(X_train, y_train)
GBR_N_y_pred = GBR_N.predict(X_test)
print('Normal - GBR Regressor')
model_error(y_test,GBR_N_y_pred)

#### Feature Importance
In this section, Transaction4, and SumTransactions have great relevance in the prediction. With this in mind, creating other groups to expand the combinations to improve the model is possible.

In [ ]:
feature_importance(GBR_G, X_train)

In [ ]:
feature_importance(GBR_N, X_train)

#### Export Results

In [ ]:
GBR_G_y_pred_f = GBR_G.predict(t1)
GBR_N_y_pred_f = GBR_N.predict(t1)
export_result(GBR_G_y_pred_f,df_test,"Test15 GBR - Gridsearch")
export_result(GBR_N_y_pred_f,df_test,"Test15 GBR - Normal")

#### Graph Error

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(16, 4))
axes[0].scatter(y_test,GBR_G_y_pred)
axes[1].scatter(y_test,GBR_N_y_pred)
axes[0].set_title('Gradient Boosting Regressor - Grid search', fontsize=12)
axes[1].set_title('Gradient Boosting Regressor - Normal', fontsize=12)
axes[0].set_xlabel('y_test', fontsize=12)
axes[0].set_ylabel('y_pred', fontsize=12)
axes[1].set_xlabel('y_test', fontsize=12)
axes[1].set_ylabel('y_pred', fontsize=12)
fig.tight_layout()

### XGB Regressor  <a class="anchor" id="8.5"></a>

XGBoost is also know as custom tree building algorithm. It is sequential model. Each sub tree is dependent on the outcome of the previous tree.
XGBoost, which stands for Extreme Gradient Boosting, is a scalable, distributed gradient-boosted decision tree (GBDT) machine learning library. It provides parallel tree boosting and is the leading machine learning library for regression, classification, and ranking problems.

##### Initial Parameters

Firstly, it is necessary to identify what are the best hyperparameter to improve our model.

In [ ]:
param_grid_XGB = {   'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 500],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0.1, 1, 10]}
XGB = XGBRegressor()

#### Randomized Search CV

In [ ]:
bestest_XGB = randomized_search_cv(XGB, param_grid_XGB, X_train, y_train)
XGB_R = bestest_XGB
XGB_R.fit(X_train, y_train)
XGB_R_y_pred = XGB_R.predict(X_test)

In [ ]:
print('Randomized Search - XGB Regression')
model_error(y_test,XGB_R_y_pred) 

#### Normal Execution

In [ ]:
XGB_N = XGB.fit(X_train, y_train)
XGB_N_y_pred = XGB_N.predict(X_test)
print('Normal - XGB Regressor')
model_error(y_test,XGB_N_y_pred)

#### Feature Importance

In [ ]:
feature_importance(XGB_N, X_train)

In [ ]:
feature_importance(XGB_R, X_train)

#### Export Results

In [ ]:
XGB_R_y_pred_f = XGB_R.predict(t1)
XGB_N_y_pred_f = XGB_N.predict(t1)
export_result(XGB_R_y_pred_f,df_test,"Test18 XGB - Randomized")
export_result(XGB_N_y_pred_f,df_test,"Test18 XGB - Normal")

#### Graph Error

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(16, 4))
axes[0].scatter(y_test,XGB_R_y_pred)
axes[1].scatter(y_test,XGB_N_y_pred)
axes[0].set_title('XGB Regressor - Grid search', fontsize=12)
axes[1].set_title('XGB Regressor - Normal', fontsize=12)
axes[0].set_xlabel('y_test', fontsize=12)
axes[0].set_ylabel('y_pred', fontsize=12)
axes[1].set_xlabel('y_test', fontsize=12)
axes[1].set_ylabel('y_pred', fontsize=12)
fig.tight_layout()